In [18]:
import numpy as np
import os
print(os.getcwd())

# If your loaded arrays have only 10,000 test cases, set num_test_cases accordingly
num_test_cases = 10000
dim0 = 16

test_cases_act = np.load("../test_cases_act.npy")
test_cases_w = np.load("../test_cases_w.npy")

# Ensure that num_test_cases does not exceed the size of the loaded arrays
num_test_cases = min(num_test_cases, test_cases_act.shape[0], test_cases_w.shape[0])

for i in range(num_test_cases):
    in1 = test_cases_act[i]
    in2 = test_cases_w[i]
    # Process the test cases as needed


/root/jupyter_notebooks/getting_started/matmul_optimized


In [19]:
in1_py = np.resize(in1, (16,16))
in2_py = np.resize(in2, (16,16))

In [20]:
import time
start_time = time.time()
out_py = np.dot(in1_py,in2_py)
end_time = time.time()
python_time = end_time - start_time
print("SW matrix mul exe time: {}s".format(end_time - start_time))

SW matrix mul exe time: 0.0010900497436523438s


In [21]:
#print("the result of software matmul: ", out_py)

In [ ]:
from pynq import Overlay
overlay_baseline = Overlay("matmult_optimized.bit")
baseline = overlay_baseline.matmul_optimized_0

In [ ]:
from pynq import allocate
input1_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')
input2_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')
output_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')

In [ ]:
np.copyto(input1_buffer0, in1.flatten())
np.copyto(input2_buffer0, in2.flatten())

#print("input1_buffer0:", input1_buffer0)
#print("input2_buffer0:", input2_buffer0)
#print("output_buffer0:", output_buffer0)

In [ ]:
input1_buffer0.sync_to_device()
input2_buffer0.sync_to_device()

#print("input1_buffer0:", input1_buffer0)
#print("input2_buffer0:", input2_buffer0)

In [ ]:
baseline.register_map

In [ ]:
baseline.register_map.B_1.address

In [ ]:
# Write A address (low and high)
baseline.write(baseline.register_map.A_1.address, input1_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.A_2.address, (input1_buffer0.physical_address >> 32) & 0xFFFFFFFF)

# Write B address (low and high)
baseline.write(baseline.register_map.B_1.address, input2_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.B_2.address, (input2_buffer0.physical_address >> 32) & 0xFFFFFFFF)

# Write AB address (low and high)
baseline.write(baseline.register_map.AB_1.address, output_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.AB_2.address, (output_buffer0.physical_address >> 32) & 0xFFFFFFFF)

In [ ]:
input1_buffer0.sync_to_device()
input2_buffer0.sync_to_device()

# Start the IP
baseline.write(0x00, 0x01)

# Wait for AP_DONE
while (baseline.read(0x00) & 0x2) == 0:
    pass

output_buffer0.sync_from_device()

In [ ]:
baseline.write(0x00, 0x01)
start_time = time.time()
while True:
    reg = baseline.read(0x00)
    if reg != 1:
        break
end_time = time.time()
baseline_time = end_time - start_time
print("HW mul (baseline) exe time: {}s".format(end_time - start_time))

In [ ]:
output_buffer0.sync_from_device()
#print("output_buffer0:", output_buffer0)

In [ ]:
#output_buffer0

In [ ]:
out_hw = output_buffer0.reshape((dim0, dim0))

#print("Hardware result:\n", out_hw)
#print("Software result:\n", out_py)

# Compare results
diff = np.abs(out_hw - out_py)
max_diff = np.max(diff)
print(f"Test Case {test_cases_act + 1}: Max Difference = {max_diff}")